# 13.13 Twitter Streaming API
* Streams **randomly selected** live tweets up to a **maximum of one percent of the tweets per day**
* According to https://InternetLiveStats.com
    * Approximately 8500 tweets per second
    * Nearly **750 million tweets per day**
* So Streaming API gives you **free access to approximately 7.5 million tweets/day**

## 13.13.1 Creating a Subclass of `StreamListener` 
* A stream uses a **persistent** connection to **push** tweets to your app
* Streaming rate varies tremendously, based on search criteria
* Subclass of Tweepy’s **`StreamListener`** listens for tweets
    * Notified when each new tweet (or other Twitter message) arrives
* Each message results in a call to a **`StreamListener` method**
* **Override** only the **methods you need**
* [Additional `StreamListener` methods](https://github.com/tweepy/tweepy/blob/master/tweepy/streaming.py)  

### `StreamListener` Methods
| Method&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;| Description
| :---	| :---
| `on_connect(self)` 	| Called when you successfully connect to the Twitter stream. This is for statements that should execute only if your app is connected to the stream.
| `on_status(self, status)` 	| Called when a tweet arrives—status is an object of Tweepy’s Status.
| `on_limit(self, track)` 	| Called when a limit notice arrives. This occurs if your search matches more tweets than Twitter can deliver based on its current streaming rate limits. In this case, the limit notice contains the number of matching tweets that could not be delivered.
| `on_error(self, status_code)` 	| Called in response to error codes sent by Twitter. 
| `on_timeout(self)` 	| Called if the connection times out—that is, the Twitter server is not responding.
| `on_warning(self, notice)` 	| Called if Twitter sends a disconnect warning to indicate that the connection might be closed. For example, Twitter maintains a queue of the tweets it’s pushing to your app. If the app does not read the tweets fast enough, `on_warning`’s notice argument will contain a warning message indicating that the connection will terminate if the queue becomes full. 

### Class `TweetListener` Defined in `tweetlistener.py`
```python  
# tweetlistener.py
"""tweepy.StreamListener subclass that processes tweets as they arrive."""
import tweepy
from textblob import TextBlob

class TweetListener(tweepy.StreamListener):
    """Handles incoming Tweet stream."""
```

```python  
    def __init__(self, api, limit=10):
        """Create instance variables for tracking number of tweets."""
        self.tweet_count = 0
        self.TWEET_LIMIT = limit
        super().__init__(api)  # call superclass's init

    def on_connect(self):
        """Called when your connection attempt is successful, enabling 
        you to perform appropriate application tasks at that point."""
        print('Connection successful\n')
```

```python  
    def on_status(self, status):
        """Called when Twitter pushes a new tweet to you."""
        # get the tweet text
        try:  
            tweet_text = status.extended_tweet.full_text
        except: 
            tweet_text = status.text

        print(f'Screen name: {status.user.screen_name}:')
        print(f'   Language: {status.lang}')
        print(f'     Status: {tweet_text}')

        if status.lang != 'en':
            print(f' Translated: {TextBlob(tweet_text).translate()}')

        print()
        self.tweet_count += 1  # track number of tweets processed

        # if TWEET_LIMIT is reached, return False to terminate streaming
        return self.tweet_count <= self.TWEET_LIMIT
```

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Class `TweetListener: __init__` Method 
* `limit` is the total number of tweets to process—10 by default
* Enables you to control the number of tweets to receive
* If `limit` is `None`, the stream will not terminate automatically

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Class `TweetListener: on_connect` Method 
* Called when your app successfully connects to the Twitter stream

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Class `TweetListener`: `on_status` Method 
* Called by Tweepy **when each tweet arrives**
* Second parameter receives a **Tweepy `Status` object** representing the tweet 
* If `on_status` returns `True`, the stream remains open
* When `on_status` returns `False`, **Tweepy disconnects** from the stream

* **Instructor Note:** In the published book, the last line of this method has a `<=` test that should be `<`&mdash;that's fixed in the version provided with these slides

# 13.13.2 Initiating Stream Processing
### Authenticating

In [ ]:
import tweepy

In [ ]:
import keys

In [ ]:
auth = tweepy.OAuthHandler(keys.consumer_key, 
                           keys.consumer_secret)

In [ ]:
auth.set_access_token(keys.access_token, 
                      keys.access_token_secret)

In [ ]:
api = tweepy.API(auth, wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True)

### Creating a `TweetListener` 
* Initializing with the `api` object
* 10 tweets will stream by default


In [ ]:
from tweetlistener import TweetListener

In [ ]:
tweet_listener = TweetListener(api)

### Creating a `Stream` 
* A Tweepy **`Stream`** object to manages the connection to the Twitter stream
* Passes the messages to your `TweetListener` 

In [ ]:
tweet_stream = tweepy.Stream(auth=api.auth, 
                             listener=tweet_listener)

### Starting the Tweet Stream 
* Use the `Stream` object’s **`filter` method** 
* `track` parameter specifies a list of search terms
* Streaming API returns full tweet **JSON objects** for tweets that match any of the terms, **not just in the tweet’s text, but also in @-mentions, hashtags, expanded URLs and other information**  
* Might not see search terms you’re tracking if you look only at tweets' text

* **Instructor Note:** We removed the `is_async` argument to ensure that the streamed tweets all appear below this cell in the notebook. Also, since the book was published, the Mars Rover is no longer in service. So we changed the search string here to `football`

In [ ]:
tweet_stream.filter(track=['football'])  #, is_async=True) 

### Asynchronous vs. Synchronous Streams
* `is_async=True` &mdash; initiate an **asynchronous tweet stream** 
* Code can continue executing while `StreamListener` waits for tweets 
* In IPython, can terminate an asynchronous tweet stream early:  
>```python
tweet_stream.running=False
```
* Without `is_async=True`, the stream is **synchronous** and the next In [] prompt appears **after the stream terminates**

### Other filter Method Parameters
* [Other `filter` method parameters](https://developer.twitter.com/en/docs/tweets/filter-realtime/guides/basic-stream-parameters) for refining your tweet searches


### Twitter Restrictions Note
* If you're storing tweets, Twitter requires you to delete any message or location data for which you receive a deletion message
* Your listener’s **`on_delete` method** will be called 
* [Deletion rules and message details](https://developer.twitter.com/en/docs/tweets/filter-realtime/guides/streaming-message-types)

------
&copy;1992&ndash;2020 by Pearson Education, Inc. All Rights Reserved. This content is based on Chapter 5 of the book [**Intro to Python for Computer Science and Data Science: Learning to Program with AI, Big Data and the Cloud**](https://amzn.to/2VvdnxE).

DISCLAIMER: The authors and publisher of this book have used their 
best efforts in preparing the book. These efforts include the 
development, research, and testing of the theories and programs 
to determine their effectiveness. The authors and publisher make 
no warranty of any kind, expressed or implied, with regard to these 
programs or to the documentation contained in these books. The authors 
and publisher shall not be liable in any event for incidental or 
consequential damages in connection with, or arising out of, the 
furnishing, performance, or use of these programs.                  